In [ ]:
%matplotlib                    # Pyplot 그래프 새창에서 열기 옵션

In [ ]:
%matplotlib inline             # Pyplot 그래프 새창에서 열기 '취소' 옵션

In [2]:
pd.set_option('display.max_rows',    100)      # DataFrame 데이터 확인 시 최대 표시 행 수
pd.set_option('display.max_columns', 100)      # DataFrame 데이터 확인 시 최대 표시 열 수

## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import pywt

.

.

.



## 데이터 크기 정의하기

In [3]:
NoOfData    = 180  # 정상/고장 데이터 각 180개씩 
NoOfSensor  = 3    # 가속도(Acceleration), 전압(Voltage), 전류(Current)
NoOfFeature = 10   # 특징 개수:10개 (순서: Max, Min, Mean, RMS, Variance, Skewness, Kurtosis, Crest factor, Shape factor, Impulse factor)

NoOfData, NoOfSensor, NoOfFeature

(180, 3, 10)

## Time Domain 특징값 추출 (10 features * 3 sensors = 30개씩)

In [4]:
def rms(x): # RMS 함수 정의
    return np.sqrt(np.mean(x**2))

In [5]:
# 특징데이터 크기 지정
TimeFeature_Normal   = np.zeros((NoOfSensor*NoOfFeature , NoOfData))
TimeFeature_Abnormal = np.zeros((NoOfSensor*NoOfFeature , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './SpotWeldingData/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_path2 = './SpotWeldingData/Abnormal_%d'%(i+1) # Abnormal 데이터 파일 경로
    temp_data1 = pd.read_csv(temp_path1 , sep=',' , header=None) # 임시 Normal 데이터
    temp_data2 = pd.read_csv(temp_path2 , sep=',' , header=None) # 임시 Abnormal 데이터
    
    # Time Domain 특징값 추출
    for j in range(NoOfSensor):
        
        # Normal Time Domain Feature
        TimeFeature_Normal[10*j+0, i] = np.max(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+1, i] = np.min(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+2, i] = np.mean(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+3, i] = rms(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+4, i] = np.var(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+5, i] = sp.skew(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+6, i] = sp.kurtosis(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+7, i] = np.max(temp_data1.iloc[:,j+1])/rms(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+8, i] = rms(temp_data1.iloc[:,j+1])/np.mean(np.abs(temp_data1.iloc[:,j+1]))
        TimeFeature_Normal[10*j+9, i] = np.max(temp_data1.iloc[:,j+1])/np.mean(np.abs(temp_data1.iloc[:,j+1]))
        
        # Abnormal Time Domain Feature
        TimeFeature_Abnormal[10*j+0, i] = np.max(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+1, i] = np.min(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+2, i] = np.mean(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+3, i] = rms(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+4, i] = np.var(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+5, i] = sp.skew(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+6, i] = sp.kurtosis(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+7, i] = np.max(temp_data2.iloc[:,j+1])/rms(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+8, i] = rms(temp_data2.iloc[:,j+1])/np.mean(np.abs(temp_data2.iloc[:,j+1]))
        TimeFeature_Abnormal[10*j+9, i] = np.max(temp_data2.iloc[:,j+1])/np.mean(np.abs(temp_data2.iloc[:,j+1]))
        
print(TimeFeature_Normal.shape)
print(TimeFeature_Abnormal.shape)

(30, 180)
(30, 180)


Normal, Abnormal 데이터 합치기  

In [6]:
TimeFeature = np.concatenate([TimeFeature_Normal, TimeFeature_Abnormal] , axis=1)
TimeFeature.shape

(30, 360)

.

.

.



## Frequency Domain 특징값 추출 (10 features \* 8 wavelet levels * 3 sensors = 240개씩)

In [7]:
# Wavelet options

MotherWavelet = pywt.Wavelet('haar')   # Mother wavelet (모함수) 지정
Level   = 8                            # Wavelet 분해 레벨 지정
select  = 8                            # 특징추출 영역 고주파 영역부터 개수 지정 (d1~)

In [8]:
#Frequency Domain 특징값 추출 (Wavelet Transform 기반)
FreqFeature_Normal   = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))
FreqFeature_Abnormal = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './SpotWeldingData/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_path2 = './SpotWeldingData/Abnormal_%d'%(i+1) # Abnormal 데이터 파일 경로
    temp_data1 = np.array(pd.read_csv(temp_path1 , sep=',', header=None).iloc[:,1:]) # 임시 Normal 데이터
    temp_data2 = np.array(pd.read_csv(temp_path2 , sep=',', header=None).iloc[:,1:]) # 임시 Abnormal 데이터
    Coef1      = pywt.wavedec(temp_data1, MotherWavelet, level=Level, axis=0)
    Coef2      = pywt.wavedec(temp_data2, MotherWavelet, level=Level, axis=0)
    
    # Frequency Domain 특징값 추출
    for j in range(NoOfSensor):
        
        for k in np.arange(select):
            coef1 = Coef1[Level-k]
            coef2 = Coef2[Level-k]
            
            # Normal Frequency Domain Feature
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef1[:,j])/rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef1[:,j])/np.mean(np.abs(coef1[:,j]))
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef1[:,j])/np.mean(np.abs(coef1[:,j]))
            
            # Abnormal Frequency Domain Feature
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef2[:,j])/rms(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef2[:,j])/np.mean(np.abs(coef2[:,j]))
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef2[:,j])/np.mean(np.abs(coef2[:,j]))

print(FreqFeature_Normal.shape)
print(FreqFeature_Abnormal.shape)

(240, 180)
(240, 180)


Normal, Abnormal 특징값 합치기      

In [9]:
FreqFeature = np.concatenate([FreqFeature_Normal, FreqFeature_Abnormal] , axis=1)
FreqFeature.shape

(240, 360)

.

.

.



## Feature 통합 (Time domain 30개 + Freq. domain 240 = 270개)

In [10]:
Features = np.concatenate([TimeFeature,FreqFeature] , axis=0)

print("Feature Data Size :", Features.shape)
print("= 데이터 %d개(정상/고장 각 %d개씩)가 각각 %d개의 특징값으로 구성됨"%(Features.shape[1], Features.shape[1]/2, Features.shape[0]))

Feature Data Size : (270, 360)
= 데이터 360개(정상/고장 각 180개씩)가 각각 270개의 특징값으로 구성됨


저장에 용이한 Data frame 형식으로 변환

In [11]:
FeatureData = pd.DataFrame(Features)
FeatureData

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359
0,1.351000,31.661000,31.832000,1.418300,1.053400,30.628000,0.992040,0.992420,1.059700,1.170800,1.084800,22.402000,22.267000,1.238500,1.078100,0.970730,0.974840,0.935210,1.029400,0.970730,0.899310,1.137900,1.123700,0.996160,1.545100,1.103500,0.962500,0.910530,0.918750,1.250400,0.991300,0.986440,1.042900,1.023800,1.037700,0.969980,29.735000,1.389900,1.149800,0.990920,0.964370,36.016000,26.252000,21.306000,0.980830,0.977460,1.005100,1.117700,1.857000,29.653000,...,0.751230,0.783010,0.998780,0.774040,0.765060,0.761320,0.740010,0.700000,0.717200,0.758330,0.751980,0.812550,0.883980,0.743000,0.919130,1.120300,0.978960,0.721690,0.955770,0.880980,0.675690,0.705980,0.685420,0.758710,0.738510,0.749730,0.932960,0.956890,0.851440,0.720190,0.754590,0.789740,0.668960,0.784130,0.722810,0.851440,1.051100,1.035800,1.048500,1.096000,0.931090,0.732910,1.016000,0.717950,0.853310,0.747490,0.718320,0.845460,0.848450,0.758330
1,-1.372000,-22.786000,-23.613000,-1.085600,-1.057500,-19.468000,-1.319600,-1.056400,-2.041700,-1.343200,-1.236300,-31.743000,-24.318000,-1.164100,-1.068000,-1.120300,-1.032500,-1.156200,-1.434100,-1.059000,-0.849230,-1.208600,-1.669300,-0.857830,-1.483100,-1.167100,-0.992820,-0.910930,-0.966640,-2.007300,-1.146500,-1.075500,-1.109900,-0.982720,-0.991320,-0.942340,-44.825000,-1.108000,-1.063500,-1.041400,-1.023100,-21.986000,-16.858000,-20.353000,-1.220500,-0.931120,-1.034000,-1.704000,-1.226900,-20.495000,...,-0.618130,-1.059400,-1.118100,-0.758360,-0.958040,-0.686940,-0.748260,-0.751260,-0.746390,-0.969640,-0.563160,-0.691430,-1.588100,-0.985340,-1.091500,-1.270300,-1.703300,-0.689930,-0.762850,-1.477800,-0.808840,-0.747520,-0.523530,-0.836510,-1.038400,-0.972250,-0.826420,-1.615400,-0.968140,-0.645430,-1.078100,-0.830530,-0.732930,-1.095300,-1.112100,-0.840250,-1.259400,-1.361100,-0.887740,-1.282600,-1.564900,-0.730310,-1.387000,-0.796880,-0.860070,-0.782290,-0.513800,-0.656270,-0.747890,-0.914290
2,0.011083,0.023339,0.020506,0.027215,0.016574,0.018563,0.020904,0.024480,0.029605,0.028426,0.024661,0.033513,0.012306,0.022461,0.019194,0.018180,0.024540,0.020940,0.024024,0.028154,0.023596,0.023600,0.033108,0.034488,0.013544,0.021327,0.018237,0.016548,0.020762,0.022284,0.026215,0.021437,0.031084,0.021675,0.033327,0.023667,0.016182,0.026916,0.030585,0.026303,0.027492,0.032113,0.024635,0.030892,0.034040,0.024174,0.034269,0.034543,0.020659,0.025106,...,0.032539,0.040197,0.024759,0.026745,0.031180,0.033509,0.035231,0.032491,0.031845,0.039828,0.038236,0.047341,0.041849,0.041040,0.017758,0.032460,0.035423,0.034613,0.029500,0.040297,0.032072,0.036734,0.032629,0.033043,0.032919,0.043961,0.029185,0.028382,0.030025,0.031060,0.031781,0.028832,0.029517,0.025209,0.033696,0.033155,0.035037,0.034659,0.034232,0.034018,0.029894,0.027896,0.032512,0.036554,0.031676,0.037731,0.036942,0.035927,0.035836,0.036279
3,0.426105,2.312749,2.313820,0.396240,0.388252,2.088591,0.403801,0.404898,0.381526,0.412919,0.411964,2.018345,1.811748,0.411718,0.406878,0.389891,0.389596,0.387056,0.400582,0.409709,0.358967,0.405389,0.401095,0.371662,0.417783,0.404115,0.395493,0.378264,0.377218,0.422364,0.423772,0.418255,0.366753,0.393314,0.391547,0.368730,2.391307,0.410065,0.409862,0.395840,0.393083,2.334685,1.757213,1.826360,0.362961,0.397067,0.398152,0.383145,0.416623,2.118041,...,0.296790,0.303777,0.332225,0.305055,0.308754,0.306795,0.303096,0.307891,0.321751,0.336856,0.275569,0.319886,0.319580,0.286993,0.367243,0.323312,0.331005,0.289436,0.303430,0.322642,0.302104,0.320424,0.249474,0.292953,0.291732,0.285504,0.335710,0.329737,0.321599,0.298419,0.298477,0.347776,0.319778,0.300164,0.265370,0.314739,0.325763,0.309731,0.365724,0.346014,0

추출된 특징 데이터 저장 (.csv 파일)

In [13]:
path = './ProcessedData/FeatureData'   # path = '파일 경로/저장할 파일 이름'
FeatureData.to_csv(path, sep=',', header=None , index=None)